In [6]:
import os
import json
import openai
import panel as pn
from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.agents import AgentExecutor, Tool
from langchain.memory import ConversationBufferMemory
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import MessagesPlaceholder
from langchain.schema import AIMessage, HumanMessage

# Initialize Panel extension
pn.extension()

# Load environment variables
load_dotenv(find_dotenv())

# Set up OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Load student data
with open('student3.json', 'r') as file:
    student_data = json.load(file)

class StudentAdvisor:
    def __init__(self, overall_percentage):
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
        self.tools = [
            Tool(
                name="student_data",
                func=self.get_student_data,
                description="Use this to get information about the student's performance and attendance."
            )
        ]
        self.functions = [format_tool_to_openai_function(f) for f in self.tools]
        self.model = ChatOpenAI(temperature=0.7).bind(functions=self.functions)
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", self.get_system_message(overall_percentage)),
            MessagesPlaceholder(variable_name="chat_history"),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad=lambda x: self.format_tool_steps(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.agent_executor = AgentExecutor(agent=self.chain, tools=self.tools, verbose=True, memory=self.memory)

    def get_system_message(self, overall_percentage):
        if 0 <= overall_percentage < 40:
            return "You are a helpful academic advisor for students scoring between 0-40%. These students need significant support and encouragement. Focus on basic concepts, study skills, and building confidence. Provide simple, actionable advice and be very patient and supportive."
        elif 40 <= overall_percentage < 60:
            return "You are a helpful academic advisor for students scoring between 40-60%. These students are showing potential but need guidance to improve. Focus on strengthening their understanding of key concepts, improving study techniques, and building consistency in their academic performance."
        elif 60 <= overall_percentage < 80:
            return "You are a helpful academic advisor for students scoring between 60-80%. These students are performing well but have room for improvement. Focus on refining their knowledge, introducing advanced concepts, and helping them develop critical thinking skills."
        elif 80 <= overall_percentage < 90:
            return "You are a helpful academic advisor for students scoring between 80-90%. These students are high performers. Focus on challenging them with complex problems, encouraging them to explore topics in depth, and developing their analytical and research skills."
        else:  # 90-100%
            return "You are a helpful academic advisor for students scoring above 90%. These students are exceptional performers. Focus on nurturing their talents, introducing them to college-level material, encouraging participation in academic competitions, and discussing potential career paths in their areas of strength."

    def get_student_data(self, query):
        return json.dumps(student_data)

    def format_tool_steps(self, intermediate_steps):
        messages = []
        for action, observation in intermediate_steps:
            messages.append(HumanMessage(content=action.log))
            messages.append(AIMessage(content=observation))
        return messages

    def conversation(self, user_input):
        if not user_input:
            return ""
        result = self.agent_executor({"input": user_input})
        return result['output']

def check_scores_and_trigger_chatbot():
    name = student_data['student_persona']['name']
    overall_percentage = student_data['student_persona']['overall_percentage']
    low_performing_subjects = []
    
    for subject in ['Physics', 'Chemistry', 'Biology']:
        score = int(student_data[f'subject_scores_{subject.lower()}']['score'].split()[0])
        if score < 60:  # Considering scores below 60 as areas needing improvement
            low_performing_subjects.append((subject, score))
    
    if 0 <= overall_percentage < 40:
        category = "0-40%"
    elif 40 <= overall_percentage < 60:
        category = "40-60%"
    elif 60 <= overall_percentage < 80:
        category = "60-80%"
    elif 80 <= overall_percentage < 90:
        category = "80-90%"
    else:
        category = "above 90%"

    initial_message = f"Hi {name}, your overall score is {overall_percentage}%. You're in the {category} category.\n "

    if low_performing_subjects:
        initial_message += "I've noticed you might need some extra support in these subjects:\n"
        for subject, score in low_performing_subjects:
            initial_message += f"- {subject} (Score: {score})\n"
    else:
        initial_message += "You're performing consistently across all subjects. "

    initial_message += "\nWhat specific area of your studies would you like to discuss or improve?\n"
    
    return initial_message, overall_percentage

# Initialize the StudentAdvisor
initial_message, overall_percentage = check_scores_and_trigger_chatbot()
advisor = StudentAdvisor(overall_percentage)

# Create the dashboard
chat_history = pn.widgets.TextAreaInput(value=initial_message, height=300, read_only=True)
user_input = pn.widgets.TextInput(placeholder="Enter your message here...")

def update_chat(event):
    if event.new:
        user_message = f"\nStudent: {event.new}\n\n"
        chat_history.value += user_message
        
        response = advisor.conversation(event.new)
        bot_message = f"Advisor: {response}\n\n"
        chat_history.value += bot_message
        
        user_input.value = ""  # Clear the input field

user_input.param.watch(update_chat, 'value')

dashboard = pn.Column(
    pn.pane.Markdown(f"# Student Advisor Chatbot (for {overall_percentage}% scoring students)"),
    chat_history,
    user_input
)

# Display the dashboard
dashboard.servable()

> Entering new AgentExecutor chain... 
Hello! How can I assist you today? 
 
> Finished chain.

Column
    [0] Markdown(str)
    [1] TextAreaInput(height=300, value="Hi Rohit, your o...)
    [2] TextInput(placeholder='Enter your message h...)